In [32]:
!pip -q install cadquery trimesh plotly

In [33]:
import cadquery as cq
from cadquery import exporters
import os

# -------------------------
# Parameters (mm)
# -------------------------
pitch   = 2.5
thread_len = 28.0
major_d = 20.0

head_d  = 28.0
head_h  = 4.0

shaft_start_d = 16.0
tip_d         = 2.0

thread_height = 1.5
thread_depth  = 1.2
thread_width  = 0.9 * pitch

# -------------------------
# 1) Head
# -------------------------
head = cq.Workplane("XY").circle(head_d/2.0).extrude(head_h)

# -------------------------
# 2) Tapered body (loft)
# -------------------------
r0 = shaft_start_d / 2.0
r1 = tip_d / 2.0

base_circle = cq.Workplane("XY").workplane(offset=head_h).circle(r0)
tip_circle  = cq.Workplane("XY").workplane(offset=head_h + thread_len).circle(r1)

body_solid = cq.Solid.makeLoft([base_circle.val(), tip_circle.val()], ruled=False)
body = cq.Workplane("XY").newObject([body_solid])

screw_body = head.union(body)

# -------------------------
# 3) Thread as "solid" ridge (not spring tube)
# -------------------------
helix_r = r0 * 0.98
helix = cq.Wire.makeHelix(pitch=pitch, height=thread_len, radius=helix_r)

profile = (
    cq.Workplane("YZ")
    .center(0, helix_r)
    .polyline([
        (-thread_depth, -thread_width/2.0),
        ( thread_height, 0.0),
        (-thread_depth,  thread_width/2.0),
    ])
    .close()
)

# sweep
thread_wp = profile.sweep(helix, isFrenet=True)

# ★ solid化
try:
    thread_wp = thread_wp.solid()
except Exception:
    pass

thread_shape = thread_wp.val()

# Z位置をヘッド上面へ
thread_shape = thread_shape.translate((0, 0, head_h))

# -------------------------
# 4) Union
# -------------------------
body_shape = screw_body.val()
screw_shape = body_shape.fuse(thread_shape)

print("body type:", type(body_shape), "valid:", body_shape.isValid())
print("thread type:", type(thread_shape), "valid:", thread_shape.isValid())
print("screw type:", type(screw_shape), "valid:", screw_shape.isValid())

# -------------------------
# 5) Export STL
# -------------------------
out = "screw_head_taper_thread.stl"
exporters.export(screw_shape, out, tolerance=0.1, angularTolerance=0.3)

print("exists?", os.path.exists(out))
if os.path.exists(out):
    print("size bytes:", os.path.getsize(out))
    print("Wrote", out)
else:
    raise FileNotFoundError("STL not created")

body type: <class 'cadquery.occ_impl.shapes.Compound'> valid: True
thread type: <class 'cadquery.occ_impl.shapes.Solid'> valid: True
screw type: <class 'cadquery.occ_impl.shapes.Compound'> valid: True
exists? True
size bytes: 2747084
Wrote screw_head_taper_thread.stl


In [34]:
import trimesh, numpy as np

m = trimesh.load("screw_head_taper_thread.stl")
V = m.vertices

mins = V.min(axis=0)
maxs = V.max(axis=0)
size = maxs - mins

print("bbox min:", mins)
print("bbox max:", maxs)
print("size xyz:", size)

# Z=0〜head_h付近
head_h = 4.0
mask = (V[:,2] >= -0.2) & (V[:,2] <= head_h + 0.2)
pts = V[mask]
r = np.sqrt(pts[:,0]**2 + pts[:,1]**2)

print("points near head:", pts.shape[0])
print("r_max near head:", r.max() if pts.shape[0] else None)

bbox min: [-1.49999976 -1.4999392   3.84000182]
bbox max: [ 1.49998105  1.50002551 34.08999634]
size xyz: [ 2.99998081  2.99996471 30.24999452]
points near head: 12
r_max near head: 1.2000147447868408


In [35]:
import trimesh, plotly.graph_objects as go
m = trimesh.load("screw_head_taper_thread.stl")
V = m.vertices
E = m.edges_unique
xs, ys, zs = [], [], []
for a,b in E:
    xs += [V[a,0], V[b,0], None]
    ys += [V[a,1], V[b,1], None]
    zs += [V[a,2], V[b,2], None]
fig = go.Figure([
    go.Mesh3d(x=V[:,0], y=V[:,1], z=V[:,2], i=m.faces[:,0], j=m.faces[:,1], k=m.faces[:,2], opacity=0.9),
    go.Scatter3d(x=xs, y=ys, z=zs, mode="lines", line=dict(width=1)),
])
fig.update_layout(scene_aspectmode="data", margin=dict(l=0,r=0,t=0,b=0))
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [36]:
import cadquery as cq
from cadquery import exporters
import os

# -------------------------
# Parameters (mm)
# -------------------------
pitch = 2.5
L = 28.0

head_d, head_h = 28.0, 4.0
shaft_start_d = 16.0
tip_d = 2.0

# 溝（ねじ山の“谷”）
groove_r = 0.55          # 溝の太さ（半径）
# 溝中心線の半径（軸表面付近）
r0 = shaft_start_d/2.0
helix_r = r0 * 0.90

# -------------------------
# Body (head + taper)
# -------------------------
head = cq.Workplane("XY").circle(head_d/2).extrude(head_h)

base = cq.Workplane("XY").workplane(offset=head_h).circle(r0)
tip  = cq.Workplane("XY").workplane(offset=head_h+L).circle(tip_d/2)

body_solid = cq.Solid.makeLoft([base.val(), tip.val()], ruled=False)
body = cq.Workplane("XY").newObject([body_solid])

body_shape = head.val().fuse(body.val())

# -------------------------
# Helical groove cutter (tube swept along helix)
# -------------------------
helix = cq.Wire.makeHelix(pitch=pitch, height=L, radius=helix_r)

# 開始点(helix_r,0,0)に円断面を置いて sweep → チューブ状カッター
cutter = cq.Workplane("XY").center(helix_r, 0).circle(groove_r).sweep(helix, isFrenet=True)
cutter_shape = cutter.val().translate((0, 0, head_h))  # ヘッド上面から開始

# -------------------------
# Cut groove into body
# -------------------------
screw_shape = body_shape.cut(cutter_shape)

out = "screw_with_groove.stl"
exporters.export(screw_shape, out, tolerance=0.1, angularTolerance=0.3)

print("Wrote", out, "exists?", os.path.exists(out), "bytes", os.path.getsize(out) if os.path.exists(out) else None)

Wrote screw_with_groove.stl exists? True bytes 2513484


In [37]:
import trimesh
import plotly.graph_objects as go

m = trimesh.load("screw_with_groove.stl")
V = m.vertices
E = m.edges_unique

xs, ys, zs = [], [], []
for a,b in E:
    xs += [V[a,0], V[b,0], None]
    ys += [V[a,1], V[b,1], None]
    zs += [V[a,2], V[b,2], None]

fig = go.Figure([
    go.Mesh3d(x=V[:,0], y=V[:,1], z=V[:,2], i=m.faces[:,0], j=m.faces[:,1], k=m.faces[:,2], opacity=0.9),
    go.Scatter3d(x=xs, y=ys, z=zs, mode="lines", line=dict(width=1)),
])
fig.update_layout(scene_aspectmode="data", margin=dict(l=0,r=0,t=0,b=0))
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [38]:
import cadquery as cq
from cadquery import exporters
import os

pitch = 2.5
L = 28.0

head_d, head_h = 28.0, 4.0
shaft_start_d = 16.0
tip_d = 2.0

r0 = shaft_start_d/2.0

# ★溝強め
groove_r = 1.5
helix_r  = r0 - groove_r*0.6

# Body
head = cq.Workplane("XY").circle(head_d/2).extrude(head_h)
base = cq.Workplane("XY").workplane(offset=head_h).circle(r0)
tip  = cq.Workplane("XY").workplane(offset=head_h+L).circle(tip_d/2)
body_solid = cq.Solid.makeLoft([base.val(), tip.val()], ruled=False)
body_shape = head.val().fuse(body_solid)

# Helix cutter
helix = cq.Wire.makeHelix(pitch=pitch, height=L, radius=helix_r)
cutter = cq.Workplane("XY").center(helix_r, 0).circle(groove_r).sweep(helix, isFrenet=True)
cutter_shape = cutter.val().translate((0, 0, head_h))

# Cut
screw_shape = body_shape.cut(cutter_shape)

out = "screw_with_groove_deep.stl"
exporters.export(screw_shape, out, tolerance=0.08, angularTolerance=0.3)
print("Wrote", out, "bytes", os.path.getsize(out))

Wrote screw_with_groove_deep.stl bytes 1450184


In [39]:
import trimesh
import plotly.graph_objects as go

m = trimesh.load("screw_with_groove_deep.stl")
V = m.vertices
E = m.edges_unique

xs, ys, zs = [], [], []
for a,b in E:
    xs += [V[a,0], V[b,0], None]
    ys += [V[a,1], V[b,1], None]
    zs += [V[a,2], V[b,2], None]

fig = go.Figure([
    go.Mesh3d(x=V[:,0], y=V[:,1], z=V[:,2], i=m.faces[:,0], j=m.faces[:,1], k=m.faces[:,2], opacity=0.9),
    go.Scatter3d(x=xs, y=ys, z=zs, mode="lines", line=dict(width=1)),
])
fig.update_layout(scene_aspectmode="data", margin=dict(l=0,r=0,t=0,b=0))
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [40]:
import cadquery as cq
from cadquery import exporters
import os

pitch = 2.5
L = 28.0
segments = 8

head_d, head_h = 28.0, 4.0
shaft_start_d = 16.0
tip_d = 2.0

r_root = shaft_start_d/2.0
r_tip  = tip_d/2.0

groove_r = 1.2  # 溝の太さ

def lerp(a,b,t): return a + (b-a)*t

# Body
head = cq.Workplane("XY").circle(head_d/2).extrude(head_h)
base = cq.Workplane("XY").workplane(offset=head_h).circle(r_root)
tip  = cq.Workplane("XY").workplane(offset=head_h+L).circle(r_tip)
body_solid = cq.Solid.makeLoft([base.val(), tip.val()], ruled=False)
shape = head.val().fuse(body_solid)

# Piecewise cut (taper-following)
for i in range(segments):
    t1 = i/segments
    t2 = (i+1)/segments
    z1 = head_h + L*t1
    z2 = head_h + L*t2
    h  = z2 - z1

    r_axis = lerp(r_root, r_tip, (t1+t2)/2)
    helix_r = r_axis - groove_r*0.6   # ← 常食い込むように内側

    helix = cq.Wire.makeHelix(pitch=pitch, height=h, radius=helix_r)
    cutter = cq.Workplane("XY").center(helix_r, 0).circle(groove_r).sweep(helix, isFrenet=True)
    shape = shape.cut(cutter.val().translate((0,0,z1)))

out = "screw_with_groove_tapered.stl"
exporters.export(shape, out, tolerance=0.08, angularTolerance=0.3)
print("Wrote", out, "bytes", os.path.getsize(out))

Wrote screw_with_groove_tapered.stl bytes 1795784


In [41]:
import trimesh
import plotly.graph_objects as go

m = trimesh.load("screw_with_groove_tapered.stl")
V = m.vertices
E = m.edges_unique

xs, ys, zs = [], [], []
for a,b in E:
    xs += [V[a,0], V[b,0], None]
    ys += [V[a,1], V[b,1], None]
    zs += [V[a,2], V[b,2], None]

fig = go.Figure([
    go.Mesh3d(x=V[:,0], y=V[:,1], z=V[:,2], i=m.faces[:,0], j=m.faces[:,1], k=m.faces[:,2], opacity=0.9),
    go.Scatter3d(x=xs, y=ys, z=zs, mode="lines", line=dict(width=1)),
])
fig.update_layout(scene_aspectmode="data", margin=dict(l=0,r=0,t=0,b=0))
fig.show()

Output hidden; open in https://colab.research.google.com to view.